In [111]:
import pandas as pd
from pathlib import Path

# Variants Output File Tour

This notebook gives a short overview on how to use the `variants_output.tsv` file.

It represents a simplified version of the full pipeline output (`output/release/built_final.tsv`) with the goal to be more easily consumable for an analyst or a downstream application. That is,
 * fields are renamed for better consistency and understandability
 * redunant or too specific fields are removed


## Getting Started
To get started, download a recent data release tar ball from https://brcaexchange.org/releases and execute the command below to unpack to content to `/tmp`

In [112]:
#!tar zxf [path to tar.gz file] -C /tmp

In [113]:
# tar_ball_out = Path('.')

In [114]:
def read_tsv(path):
    return pd.read_csv(path, sep='\t', na_values='-', low_memory=False)

## Explore


In case a field is of interest to you, but not included in the `variants_output.tsv` file, you can retrieve it from the full output file as lined out below.


In [115]:
full_output_path = 'built_final.tsv'

In [116]:
df_full = read_tsv(full_output_path)

In [117]:
df_full.shape


(72467, 470)

In [118]:
df_brca1 = df_full[
    (df_full["Gene_Symbol"] == "BRCA1") &
    (df_full["Genomic_Coordinate_hg38"].notna()) &
    (df_full["Clinical_significance_ENIGMA"].isin(["Benign", "Likely benign", "Pathogenic"]))
]



In [119]:
cols = ["Chr", "Ref", "Alt", "Clinical_significance_ENIGMA", "Genomic_Coordinate_hg38",
        "Hg38_Start",
        "Hg38_End",]


In [120]:
from collections import Counter
Counter(df_brca1["Clinical_significance_ENIGMA"])

Counter({'Pathogenic': 2232, 'Benign': 668, 'Likely benign': 441})

In [121]:
df_brca1[cols].shape


(3341, 7)

In [122]:
df_brca1 = df_brca1[cols]

In [123]:
df_brca1
# BRCA location 43044295..43170327

,Chr,Ref,Alt,Clinical_significance_ENIGMA,Genomic_Coordinate_hg38,Hg38_Start,Hg38_End
36870,17,G,A,Benign,chr17:43039471:G>A,43039471,43039471
36871,17,G,A,Benign,chr17:43039818:G>A,43039818,43039818
36872,17,C,CAT,Benign,chr17:43039999:C>CAT,43039999,43039999
36873,17,C,T,Benign,chr17:43040165:C>T,43040165,43040165
36874,17,C,G,Benign,chr17:43041129:C>G,43041129,43041129
...,...,...,...,...,...,...,...
72462,17,G,C,Benign,chr17:43127544:G>C,43127544,43127544
72463,17,A,G,Benign,chr17:43127753:A>G,43127753,43127753
72464,17,C,A,Benign,chr17:43127820:C>A,43127820,43127820
72465,17,A,G,Benign,chr17:43127865:A>G,43127865,43127865


In [124]:
with open("outxxx.txt", "w") as f:
    for i in df_full.head().columns:
        print(i, file=f)

# Extract BRCA Gene

In [125]:
from Bio import SeqIO

In [126]:
# BRCA location 43044295..43170327

fasta_file = "hg38_chr17.fasta"

start = 43044295
end = 43170327

with open(fasta_file, "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        # if record.id == "chr17":
        # Extract the sequence (convert to 0-based indexing for Python slicing)
        full = record.seq
        extracted_sequence = record.seq[start-1:end]
        # reverse_complement = extracted_sequence.reverse_complement()
        # print(f"Extracted sequence ({start}-{end}):")
        print(extracted_sequence)
print(len(extracted_sequence))

...
126033


In [127]:
# brca_genes = [extracted_sequence]

In [128]:
# ctr = 0


brca_mutations = []

for row in df_brca1.itertuples(index=True, name='Row'):
    
    if row.Hg38_Start > 43044295:
        # ctr += 1
        
        if len(row.Ref) == len(row.Alt):
            brca_start = row.Hg38_Start-start
            brca_end = row.Hg38_End-start
            
            print(extracted_sequence[brca_start:brca_end+1] , row.Ref, row.Alt)
            
            assert extracted_sequence[brca_start:brca_end+1] == row.Ref
            
            mutated = extracted_sequence[:brca_start] + row.Alt + extracted_sequence[brca_end+1:]
            
            assert (len(extracted_sequence) - len(mutated))== (len(row.Ref) - len(row.Alt))
            
            brca_mutations.append(str(mutated))
            
        else:
            brca_mutations.append(None)
        
    else:
        brca_mutations.append(None)
        

# print(ctr)
# print(full[43127820-1])
# print(43127820-start)
# extracted_sequence[43127820-start]


C C T
C C T
G G A
C C T
G G A
C C A
T T C
G G C
G G A
G G C
G G A
T T G
G G T
T T G
C C T
G G C
G G T
G G C
C C G
G G A
G G T
C C T
G G A
G G C
G G T
G G A
A A C
A A G
A A G
C C A
A A T
C C T
C C T
A A C
A A G
C C A
G G A
C C T
C C T
G G T
T T G
G G A
A A G
G G C
C C T
T T C
G G A
A A G
C C A
T T C
C C T
C C T
A A G
G G A
C C T
G G A
A A T
T T C
T T C
T T C
C C T
A A G
T T A
A A G
T T G
G G A
G G A
G G C
T T C
G G T
C C A
T T A
C C A
CCACA CCACA TCACT
AC AC CT
G G A
T T C
C C T
C C T
C C A
C C T
T T C
G G A
T T C
G G T
T T C
C C T
C C T
G G A
A A T
G G A
T T G
C C T
G G A
A A C
A A T
G G A
C C G
C C T
C C A
A A T
G G A
A A T
A A C
C C A
A A G
G G T
G G C
T T A
T T C
C C T
C C T
G G A
C C G
T T C
C C T
G G A
A A G
T T C
T T C
T T C
G G A
T T G
C C T
C C T
C C T
G G A
G G A
A A G
T T C
C C T
A A G
A A T
T T C
C C G
C C A
C C T
C C T
A A C
C C T
A A G
A A G
A A G
C C T
C C T
T T C
C C T
G G A
C C A
T T C
T T G
C C T
G G A
G G A
C C A
C C T
T T A
A A G
T T C
C C T
A A G
A A T
G G A
T T C
C

In [129]:
print(len(brca_mutations))
print(len([x for x in brca_mutations if x is not None]))


3341
1521


In [130]:
def scrapeCDSfromGenbank(filename):
    """Scrapes CDS feature ranges from a GenBank file.

    Args:
        filename (string): Path to the GenBank (.gb) file.

    Returns:
        list[(int, int)]: List of ranges (start, end) for all CDS features.
    """
    cds_ranges = []

    # Parse the GenBank file
    with open(filename, "r") as handle:
        for record in SeqIO.parse(handle, "genbank"):
            for feature in record.features:
                if feature.type == "CDS":
                    start = int(feature.location.start)
                    end = int(feature.location.end)
                    cds_ranges.append((start, end))

    return cds_ranges

In [131]:
print(scrapeCDSfromGenbank("brca1_hg38_chr17.gb")[0])

(1383, 79802)


In [132]:
def extract_exon_locations(filename):
    """Extracts exon locations (joined intervals) from a GenBank file.

    Args:
        filename (string): Path to the GenBank (.gb) file.

    Returns:
        list[list[tuple[int, int]]]: A list of exon ranges for each CDS or exon feature.
                                     Each feature is represented as a list of (start, end) tuples.
    """
    exon_locations = {}

    # Parse the GenBank file
    with open(filename, "r") as handle:
        for record in SeqIO.parse(handle, "genbank"):
            for feature in record.features:
                # Look for CDS or exon features
                if feature.type in ["CDS", "exon"]:
                    if "join" in str(feature.location):  # Handles joined intervals
                        # Extract all parts of the join
                        exon_ranges = [
                            (int(part.start+1), int(part.end))
                            for part in feature.location.parts
                        ]
                        # print(feature.qualifiers.get("protein_id", []))
                    else:
                        # Single interval
                        exon_ranges = [(int(feature.location.start), int(feature.location.end))]

                    
                    exon_locations[feature.qualifiers.get("protein_id", [])[0]] = exon_ranges[::-1]
    
    return exon_locations


In [133]:
print(extract_exon_locations("brca1_hg38_chr17.gb")["NP_001394531.1"])

[(1384, 1508), (3349, 3409), (4827, 4900), (6769, 6823), (12758, 12841), (19039, 19079), (19580, 19657), (23314, 23401), (26634, 26944), (30037, 30227), (32194, 32320), (38110, 38281), (46650, 46738), (47141, 50566), (51552, 51628), (52950, 52995), (55481, 55586), (59828, 59967), (60574, 60662), (62162, 62239), (71432, 71485), (79723, 79802)]


In [134]:
def reconstruct_cds(seq, exon_ranges):
    """Reconstructs the coding sequence (CDS) from the genomic sequence and exon ranges.

    Args:
        seq (str): The full genomic sequence as a string.
        exon_ranges (list[tuple[int, int]]): List of exon ranges (start, end) as 0-based coordinates.

    Returns:
        str: The reconstructed CDS sequence.
    """
    
    if seq is None:
        return None
    cds = ""
    for start, end in exon_ranges:
        cds += seq[start-1:end]  # Extract each exon and concatenate
    return cds

In [135]:
exons = extract_exon_locations("brca1_hg38_chr17.gb")

# print(exons["NP_001394531.1"], len(extracted_sequence))

cds = reconstruct_cds(extracted_sequence, exons["NP_001394531.1"])

# cds[1500-1384:1508-1384]
cds

Seq('TCAGTAGTGGCTGTGGGGGATCTGGGGTATCAGGTAGGTGTCCAGCTCCTGGCA...CAT')

In [136]:
from Bio.Seq import Seq

def transcribe_and_translate(cds):
    """Transcribes a DNA CDS to RNA and translates it to a protein sequence.

    Args:
        cds (str): Coding DNA sequence (CDS).

    Returns:
        tuple: Transcribed mRNA sequence and translated protein sequence.
    """
    
    if cds is None:
        return None
    # Convert to Seq object
    dna_seq = Seq(cds)
    
    # print(cds)
    
    dna_seq = dna_seq.reverse_complement()
    
    # print(dna_seq)

    # Transcribe (DNA → RNA)
    rna_seq = dna_seq.transcribe()
    # print(f"Transcribed mRNA: {rna_seq}")

    # Translate (RNA → Protein)
    protein_seq = rna_seq.translate(to_stop=True)  # Stop translation at stop codon
    # print(f"Translated Protein: {protein_seq}")

    return str(protein_seq)

# Example CDS
# cds = "ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG"
# rna_seq, protein_seq = transcribe_and_translate(cds)


In [137]:
len(transcribe_and_translate(cds))

1863

In [138]:
mutated_proteins = [transcribe_and_translate(reconstruct_cds(x, exons["NP_001394531.1"])) for x in brca_mutations]

df_brca1["seq"] = mutated_proteins

df_brca1 = df_brca1[df_brca1["seq"].notna()]

df_brca1.to_csv('brca_mutations.csv', index=False)

df_brca1


,Chr,Ref,Alt,Clinical_significance_ENIGMA,Genomic_Coordinate_hg38,Hg38_Start,Hg38_End,seq
36903,17,C,T,Benign,chr17:43044346:C>T,43044346,43044346,MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKF...
36904,17,C,T,Benign,chr17:43044351:C>T,43044351,43044351,MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKF...
36921,17,G,A,Benign,chr17:43044391:G>A,43044391,43044391,MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKF...
36974,17,C,T,Benign,chr17:43044565:C>T,43044565,43044565,MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKF...
37133,17,G,A,Benign,chr17:43044897:G>A,43044897,43044897,MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKF...
...,...,...,...,...,...,...,...,...
72461,17,C,T,Benign,chr17:43127517:C>T,43127517,43127517,MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKF...
72462,17,G,C,Benign,chr17:43127544:G>C,43127544,43127544,MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKF...
72463,17,A,G,Benign,chr17:43127753:A>G,43127753,43127753,MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKF...
72464,17,C,A,Benign,chr17:43127820:C>A,43127820,43127820,MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKF...


In [109]:
mutated_proteins = [transcribe_and_translate(reconstruct_cds(x, exons["NP_001394531.1"])) for x in brca_mutations]


Counter({None: 1820,
         'MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKFCMLKLLNQKKGPSQCPLCKNDITKRSLQESTRFSQLVEELLKIICAFQLDTGLEYANSYNFAKKENNSPEHLKDEVSIIQSMGYRNRAKRLLQSEPENPSLQETSLSVQLSNLGTVRTLRTKQRIQPQKTSVYIELGSDSSEDTVNKATYCSVGDQELLQITPQGTRDEISLDSAKKAACEFSETDVTNTEHHQPSNNDLNTTEKRAAERHPEKYQGSSVSNLHVEPCGTNTHASSLQHENSSLLLTKDRMNVEKAEFCNKSKQPGLARSQHNRWAGSKETCNDRRTPSTEKKVDLNADPLCERKEWNKQKLPCSENPRDTEDVPWITLNSSIQKVNEWFSRSDELLGSDDSHDGESESNAKVADVLDVLNEVDEYSGSSEKIDLLASDPHEALICKSERVHSKSVESNIEDKIFGKTYRKKASLPNLSHVTENLIIGAFVTEPQIIQERPLTNKLKRKRRPTSGLHPEDFIKKADLAVQKTPEMINQGTNQTEQNGQVMNITNSGHENKTKGDSIQNEKNPNPIESLEKESAFKTKAEPISSSISNMELELNIHNSKAPKKNRLRRKSSTRHIHALELVVSRNLSPPNCTELQIDSCSSSEEIKKKKYNQMPVRHSRNLQLMEGKEPATGAKKSNKPNEQTSKRHDSDTFPELKLTNAPGSFTKCSNTSELKEFVNPSLPREEKEEKLETVKVSNNAEDPKDLMLSGERVLQTERSVESSSISLVPGTDYGTQESISLLEVSTLGKAKTEPNKCVSQCAAFENPKGLIHGCSKDNRNDTEGFKYPLGHEVNHSRETSIEMEESELDAQYLQNTFKVSKRQSFAPFSNPGNAEEECATFSAHSGSLKKQSPKVTFECEQKEENQGKNESNIKPVQTVNITAGFPVVGQKDKPVDNAKCSIKGGSRFCLSSQFRGNETGLITPN

In [ ]:
with open("mutations.txt", "w") as f:
    for i in brca_mutations:
        print(i, file=f)

In [1]:
x = "MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKFCMLKLLNQKKGPSQCPLCKNDITKRSLQESTRFSQLVEELLKIICAFQLDTGLEYANSYNFAKKENNSPEHLKDEVSIIQSMGYRNRAKRLLQSEPENPSLQETSLSVQLSNLGTVRTLRTKQRIQPQKTSVYIELGSDSSEDTVNKATYCSVGDQELLQITPQGTRDEISLDSAKKAACEFSETDVTNTEHHQPSNNDLNTTEKRAAERHPEKYQGSSVSNLHVEPCGTNTHASSLQHENSSLLLTKDRMNVEKAEFCNKSKQPGLARSQHNRWAGSKETCNDRRTPSTEKKVDLNADPLCERKEWNKQKLPCSENPRDTEDVPWITLNSSIQKVNEWFSRSDELLGSDDSHDGESESNAKVADVLDVLNEVDEYSGSSEKIDLLASDPHEALICKSERVHSKSVESNIEDKIFGKTYRKKASLPNLSHVTENLIIGAFVTEPQIIQERPLTNKLKRKRRPTSGLHPEDFIKKADLAVQKTPEMINQGTNQTEQNGQVMNITNSGHENKTKGDSIQNEKNPNPIESLEKESAFKTKAEPISSSISNMELELNIHNSKAPKKNRLRRKSSTRHIHALELVVSRNLSPPNCTELQIDSCSSSEEIKKKKYNQMPVRHSRNLQLMEGKEPATGAKKSNKPNEQTSKRHDSDTFPELKLTNAPGSFTKCSNTSELKEFVNPSLPREEKEEKLETVKVSNNAEDPKDLMLSGERVLQTERSVESSSISLVPGTDYGTQESISLLEVSTLGKAKTEPNKCVSQCAAFENPKGLIHGCSKDNRNDTEGFKYPLGHEVNHSRETSIEMEESELDAQYLQNTFKVSKRQSFAPFSNPGNAEEECATFSAHSGSLKKQSPKVTFECEQKEENQGKNESNIKPVQTVNITAGFPVVGQKDKPVDNAKCSIKGGSRFCLSSQFRGNETGLITPNKHGLLQNPYRIPPLFPIKSFVKTKCKKNLLEENFEEHSMSPEREMGNENIPSTVSTISRNNIRENVFKEASSSNINEVGSSTNEVGSSINEIGSSDENIQAELGRNRGPKLNAMLRLGVLQPEVYKQSLPGSNCKHPEIKKQEYEEVVQTVNTDFSPYLISDNLEQPMGSSHASQVCSETPDDLLDDGEIKEDTSFAENDIKESSAVFSKSVQKGELSRSPSPFTHTHLAQGYRRGAKKLESSEENLSSEDEELPCFQHLLFGKVNNIPSQSTRHSTVATECLSKNTEENLLSLKNSLNDCSNQVILAKASQEHHLSEETKCSASLFSSQCSELEDLTANTNTQDPFLIGSSKQMRHQSESQGVGLSDKELVSDDEERGTGLEENNQEEQSMDSNLGEAASGCESETSVSEDCSGLSSQSDILTTQQRDTMQHNLIKLQQEMAELEAVLEQHGSQPSNSYPSIISDSSALEDLRNPEQSTSEKAVLTSQKSSEYPISQNPEGLSADKFEVSADSSTSKNKEPGVERSSPSKCPSLDDRWYMHSCSGSLQNRNYPSQEELIKVVDVEEQQLEESGPHDLTETSYLPRQDLEGTPYLESGISLFSDDPESDPSEDRAPESARVGNIPSSTSALKVPQLKVAESAQSPAAAHTTDTAGYNAMEESVSREKPELTASTERVNKRMSMVVSGLTPEEFMLVYKFARKHHITLTNLITEETTHVVMKTDAEFVCERTLKYFLGIAGGKWVVSYFWVTQSIKERKMLNEHDFEVRGDVVNGRNHQGPKRARESQDRKIFRGLEICCYGPFTNMPTDQLEWMVQLCGASVVKELSSFTLGTGVHPIVVVQPDAWTEDNGFHAIGQMCEAPVVTREWVLDSVALYQCQELDTYLIPQIPHSHY"
len(x)

1863